In [1]:
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require(tidyverse)
require(ggplot2)
require(data.table)
require(egg)
require(foreach)
require(doParallel)
registerDoParallel(cores=8)

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: egg

Loading required package: gridExtra


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



In [94]:
pos_controls

[1] "Acinetobacter lwoffii"         "Aerococcus viridans"          
 [3] "Bacillus thuringiensis"        "Bacillus pumilus"             
 [5] "Bacillus stratosphericus"      "Dietzia papillomatosis"       
 [7] "Moraxella osloensis"           "Planobacterium taklimakanense"
 [9] "Prolinoborus fasciculus"       "Cutibacterium acnes"          
[11] "Cutibacterium avidum"          "Cutibacterium granulosum"     
[13] "Pseudomonas mosselii"          "Sphingomonas aurantiaca"      
[15] "Staphylococcus aureus"         "Staphylococcus caprae"        
[17] "Staphylococcus warneri"

In [93]:
pos_control_files <- list.files("data/reported_controls/", pattern = "positive")
pos_controls <- read.csv(paste0("data/reported_controls/", pos_control_files))$positive_controls

neg_control_files <- list.files("data/reported_controls/", pattern = "negative")

neg_list <- list()

for (file in neg_control_files) {
    study_name <- paste0(unlist(str_split(file, pattern = "_"))[1:2], collapse = "_")
    neg <- read.csv(paste0("data/reported_controls/", file))
    neg_list[[study_name]] <- neg$taxa
}


In [54]:
require(VennDiagram)

venn.diagram(neg_list, filename = "data/reported_controls/test.png",
#              category.names = c("All genera", "Human associated", "Kitome associated"),
             print.mode = c("raw", "percent")
             )

[1] 1

In [77]:
study_df <- as.data.frame(table(unlist(neg_list))) %>%
    filter(Freq >= 2)

neg_shortlist <- study_df$Var1

In [78]:
max_batch_prev <- fread("results/decontamination/max_batch_prevalence.kit.site.instrument.read_t100.csv")
data_contam <- max_batch_prev %>% filter(max_batch_prev > 0.7)
data_contam <- data_contam$meta_col

In [103]:
nc <- read.csv("results/decontamination/clr_decontamination/parsed.NC.n9999.min_samples100.l10h90.txt")$non_contaminants


In [84]:
pos_controls

[1] "damgaard_2015_positive.txt"

In [104]:
venn_list <- list(nc = nc, neg_controls = neg_shortlist, pos_controls = pos_controls, prevalent_taxa = data_contam)
venn.diagram(venn_list, filename = "data/reported_controls/test2.png",
#              category.names = c("All genera", "Human associated", "Kitome associated"),
             print.mode = c("raw", "percent")
             )

[1] 1